In [250]:
import numpy as np
import pandas as pd
import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import random

from sklearn.svm import SVR



pd.options.plotting.backend = "plotly"

In [251]:
cities = pd.read_excel('cities.xlsx', index_col=0)#cities and their coordinates
cities_names = pd.read_excel('cities_names.xlsx', index_col=0)  #we will put names a bit moved from the markers
lines = pd.read_excel('partners.xlsx', index_col=0)
lines

,latitude,longitude,year
city,,,
Hamburg,53.626077,9.465599,1241.0
Lubeka,51.568180,8.106338,1241.0
Roztoka,53.900885,10.770624,1259.0
Wismar,54.001695,11.416232,1259.0
Stade,52.274361,10.522475,1253.0
Kolonia,53.069507,8.850005,1253.0
Brema,53.554957,9.989304,1253.0
Munster,51.718716,7.900764,1253.0
Dortmund,51.718716,7.900764,1253.0


In [252]:
class Alliance():
    def __init__(self, allies, year):
        self.cities = allies
        self.year = year

    def __str__(self):
        return f'{self.cities}, {self.year}'

    #returns a list of latitude coordinates
    @property
    def lat_coordinates(self):  
        lat_cords = []
        if len(self.cities) > 2:
            for ally in self.cities:
                lat_connect_point = lines.loc[ally, 'latitude']   #point on the map of which lines will connect
                lat_cords.extend([lat_connect_point, cities.loc[ally, 'latitude']])
        else:
            for ally in self.cities:
                lat_cords.append(cities.loc[ally, 'latitude'])
        return lat_cords


     #returns a list of longitude coordinates
    @property
    def lon_coordinates(self):  
        lon_cords = []
        if len(self.cities) > 2: 
            for ally in self.cities: 
                lon_connect_point = lines.loc[ally, 'longitude']   #point on the map of which lines will connect
                lon_cords.extend([lon_connect_point, cities.loc[ally, 'longitude']])
        else:
            for ally in self.cities:
                lon_cords.append(cities.loc[ally, 'longitude'])
        return lon_cords

In [253]:
d = []
with open("alliances.txt") as f:
    for line in f:
        alliance = line.split()[0]
        allies = alliance.split(',')[0:-1]
        year = alliance.split(',')[-1]
        d.append(Alliance(allies, year))
        

In [254]:
cities.loc['Hamburg', 'longitude']

9.98930408539601

In [255]:
print(d[0].cities)

['Hamburg', 'Lubeka']


In [283]:
colors = [
        'rgb(18, 220, 227)', 'rgb(251, 255, 0)', 'rgb(24, 102, 22)',
        'rgb(245, 15, 241)', 'rgb(8, 8, 153)', 'rgb(16, 222, 77)',
        'rgb(250, 0, 0)', 'rgb(250, 117, 7)', 'rgb(110, 5, 247)'
]

In [284]:
fig = go.Figure(go.Scattergeo())


fig.add_trace(go.Scattergeo(
    mode = "text",
    lon = cities_names['longitude'],
    lat = cities_names['latitude'],
    text = cities_names.index,
    textfont = {'family': 'Times New Roman'},
    showlegend = False
    ))

fig.add_trace(go.Scattergeo(   #this trace is inside loop only to have same colors as lines
    mode = "markers",
    lon = cities['longitude'],
    lat = cities['latitude'],
    marker = {'color': 'black'},   
    showlegend = False
    ))
for alliance in d:
    fig.add_trace(go.Scattergeo(
        mode = 'lines',
        lon = alliance.lon_coordinates,
        lat = alliance.lat_coordinates,
        line = {"color" : colors.pop()},
        legendgroup = d.index(alliance),
        name = alliance.year
    ))


fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'style': "carto-positron",
        'zoom': 2},
    legend = {
        'yanchor': 'middle',
        'y': 0.5,
        'xanchor': 'right',
        'x':1})